In [55]:
# Set up
# Before this setup, please go and check "README.md" to install what are necessary for running the following code.
from dolo import *

import dolark
from dolark import HModel  # The model is written in yaml file, and HModel reads the yaml file.
from dolark.equilibrium import find_steady_state
from dolark.perturbation import perturb   #find variations
from dolo import time_iteration, improved_time_iteration     #time iteration is backward induction
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

Options for the following code. Setting Fast to true will set initial capital levels to the steady state level in the solver, dramatically increasing run time.

In [56]:
Fast = True

Declare parameters that do not change. These correspond to the values chosen in Aiyagari (1994).

In [57]:
# Declare these consistent parameters.
β = 0.96
δ = 0.08
α = 0.36

Define functions that will be useful during calculation. 

In [58]:
σAdjust = lambda ρ, σ: σ*(1-ρ**2)**0.5

# We need to set steady state values of labor.
# Since we know ρ and σ we can calculate this using properties of the AR1 process and lognormal distribution:
ΣUnconditional = lambda ρ, σ: (σ**2)/(1-ρ**2)
LMean = lambda ρ, σ: np.exp(0 + ΣUnconditional(ρ, σAdjust(ρ, σ))/2)

rFunc = lambda k: α*(k)**(α - 1) - δ
wFunc = lambda k: (1-α)*(k)**(α)

In [59]:
aggmodel = HModel('Aiyagari.yaml')


load_capital = [5.718341702347233, 5.998527915377517, 6.236659640412816, 5.727020642279832, 6.023762697282315, 6.277078594476787, 5.740838016083044, 6.066502939823299, 6.348284509611548, 5.746926103372787, 6.111043099717087, 6.452912003167148, 6.2559200120869, 6.811669970608172, 7.242157096783963, 6.316337432044385, 6.969322649437888, 7.476453776919827, 6.412728832934068, 7.238541044351281, 7.89109764156157, 6.500822965861592, 7.642509470947722, 8.67333741118939]

σ_values = [0.2, 0.4]
ρ_values = [0.0, 0.3, 0.6, 0.9]
θ_values = [1, 3, 5]
results = []

index = 0
for z in σ_values:  
    for y in ρ_values:
        for x in θ_values:
            aggmodel.model.set_calibration( θ = x )  
            aggmodel.model.set_calibration( ρ_param = y ) 
            Σ_adjusted = σAdjust(y, z)**2
            aggmodel.model.set_calibration( Σ_param = Σ_adjusted )

            aggmodel.set_calibration( L = LMean(y, z) )

            if Fast == True:
                aggmodel.set_calibration( K = load_capital[index] )

            eq = find_steady_state(aggmodel)
            df = eq.as_df()
            a = df['a']
            e = df['e']
            i = df['i']
            μ = df['μ']

            # calculate ratio of K/L
            # note, we could use L = LMean(y, z) as before instead of np.inner(np.exp(e), μ)
            k = np.inner(i,μ) / np.inner(np.exp(e), μ)

            # calculate aggregate consumption using r and w
            r = rFunc(k)
            w = wFunc(k)

            c = [(1+r)*a[j] + w*np.exp(e[j]) - i[j] for j in range(len(df))]

            # calculate aggregate income using r and w
            income = [(r+δ)*a[j] + w*np.exp(e[j]) for j in range(len(df))]

            # aggregate consumption and innome
            agg_c = np.inner(c,μ)
            agg_inc = np.inner(income,μ)
            
            saving = (1 - agg_c/agg_inc)*100   #convert to %
            saving_rate = float("%.2f" % saving)  #with 2 decimals

            interest = r*100
            interest_rate = float("%.4f" % interest)  #with 4 decimals

            steady_state_capital = np.inner(a,μ)
            
            results.append([saving_rate, interest_rate, steady_state_capital])     # append the results
            index = index + 1


Computing Initial Rule... ------------------------------------------------------------------------------------------------------
| Start Improved Time Iterations.                                                                    |
------------------------------------------------------------------------------------------------------
| N    | f_x       | d_x       | Time_residuals | Time_inversion | Time_search | Lambda_0  | N_invert | N_search |
------------------------------------------------------------------------------------------------------
|    0 | 2.099e+02 | 3.148e+00 | 8.349e-03 | 1.661e-02 | 0.000e+00 | 2.539e-01 |       12 |        0 |
|    1 | 1.142e+02 | 7.152e+00 | 8.004e-03 | 1.600e-02 | 8.298e-03 | 4.714e-01 |       18 |        0 |
|    2 | 5.484e+01 | 1.154e+01 | 8.224e-03 | 3.276e-02 | 0.000e+00 | 6.011e-01 |       34 |        0 |
|    3 | 2.613e+01 | 1.749e+01 | 8.214e-03 | 4.919e-02 | 0.000e+00 | 7.403e-01 |       55 |        0 |
|    4 | 1.383e+01 | 2.885e+01 | 1.

In [60]:
aiyagari = [[23.67, 4.1666], [23.71, 4.1456], [23.83, 4.0858], [23.73, 4.1365], [23.91, 4.0432], [24.19, 3.9054], [23.82, 4.0912], [24.25, 3.8767], [24.86, 3.5857], [24.14, 3.9305], [25.51, 3.2903], [27.36, 2.5260], [23.87, 4.0649], [24.44, 3.7816], [25.22, 3.4177], [24.09, 3.9554], [25.22, 3.4188], [26.66, 2.8032], [24.5, 3.7567], [26.71, 2.7835], [29.37, 1.807], [25.47, 3.3054], [31.0, 1.2894], [37.63, -0.3456]]

conv = lambda x, y, xd, yd: ("%."+str(xd)+"f") % x + "/" + ("%."+str(yd)+"f") % y

row_names = {
    0:'ρ = 0',
    1:'ρ = 0.3',
    2:'ρ = 0.6',
    3:'ρ = 0.9'
}

def OutputTables(title, subtitle, operation_func, xd = 4, yd = 2):
    index = 0
    print(title)
    for sigma in range(1, 3):
        output = []
        for rho in range(1, 5):
            output_slice = []
            for theta in range(1, 4):
                r1, r2 = operation_func(results[index][0], results[index][1], aiyagari[index][0], aiyagari[index][1])
                if not r2 is None:
                    output_slice.append(conv(r1, r2, xd, yd))
                else:
                    output_slice.append(str(r1))
                index = index + 1
            output.append(output_slice)
        df = pd.DataFrame(output, columns=["µ = 1", "µ = 3", "µ = 5"])
        df = df.rename(index = row_names)
        print(subtitle + " (σ = " + str(0.2*sigma) + ")")
        print(df)
    print("\n")

def aiyagari_only(agg_s, r, A_agg_s, A_r):
    return A_r, A_agg_s

def results_only(agg_s, r, A_agg_s, A_r):
    return r, agg_s

OutputTables("Table II, Aiyagari (1994)", "Net return to capital in % / aggregate saving rate in %", aiyagari_only)
OutputTables("Table II, Replication (DolARK)", "Net return to capital in % / aggregate saving rate in %", results_only)

def differences(agg_s, r, A_agg_s, A_r):
    return r - A_r, agg_s - A_agg_s

OutputTables("DolARK minus Aiyagari (1994)", "Net return to capital in % / aggregate saving rate in %", differences)

def differences_percent(agg_s, r, A_agg_s, A_r):
    return 100*(r - A_r)/A_r, 100*(agg_s - A_agg_s)/A_agg_s

OutputTables("DolARK minus Aiyagari (1994), Percent Error", "Net return to capital in % / aggregate saving rate in %", differences_percent, 2, 2)


Table II, Aiyagari (1994)
Net return to capital in % / aggregate saving rate in % (σ = 0.2)
                µ = 1         µ = 3         µ = 5
ρ = 0    4.1666/23.67  4.1456/23.71  4.0858/23.83
ρ = 0.3  4.1365/23.73  4.0432/23.91  3.9054/24.19
ρ = 0.6  4.0912/23.82  3.8767/24.25  3.5857/24.86
ρ = 0.9  3.9305/24.14  3.2903/25.51  2.5260/27.36
Net return to capital in % / aggregate saving rate in % (σ = 0.4)
                µ = 1         µ = 3          µ = 5
ρ = 0    4.0649/23.87  3.7816/24.44   3.4177/25.22
ρ = 0.3  3.9554/24.09  3.4188/25.22   2.8032/26.66
ρ = 0.6  3.7567/24.50  2.7835/26.71   1.8070/29.37
ρ = 0.9  3.3054/25.47  1.2894/31.00  -0.3456/37.63


Table II, Replication (DolARK)
Net return to capital in % / aggregate saving rate in % (σ = 0.2)
                µ = 1         µ = 3         µ = 5
ρ = 0    3.9451/24.11  3.5850/24.86  3.2999/25.49
ρ = 0.3  3.9335/24.13  3.5539/24.93  3.2533/25.59
ρ = 0.6  3.9151/24.17  3.5017/25.04  3.1723/25.78
ρ = 0.9  3.9071/24.19  3.4480/25.16  3

In [11]:
df = eq.as_df()

#print(1 + df['r'][0])
#print(1 + 0.36*(1.0833/40)**(1-0.36) - 0.08)
#print(df['w'][0])
#print((1-0.36)*(40/1.0833)**(0.36))

# extract variables from the steady state solution
a = df['a']
#r = df['r']
#w = df['w']
e = df['e']
i = df['i']
μ = df['μ']

agg_i = np.inner(i,μ)
r = 0.36*(1.0833/agg_i)**(1-0.36) - 0.08
w = (1-0.36)*(agg_i/1.0833)**(0.36)

# calculate consumption
c = [(1+r)*a[j] + w*np.exp(e[j]) - i[j] for j in range(len(df))]

# calculate income 
income = [(r+0.08)*a[j] + w*np.exp(e[j]) for j in range(len(df))]   #depreciation is 8%

print(np.inner(a,μ))
print("aggregate L", np.inner(np.exp(e), μ))
print("aggregate K", np.inner(i, μ))

agg_c = np.inner(c,μ)
agg_inc = np.inner(income,μ)
print("aggregate cons", agg_c)
print("aggregate inc", agg_inc)

# saving rate 
saving = 1 - agg_c/agg_inc
print("R", 1 + r)
print("agg saving rate", saving*100)

6.412728832934068
aggregate L 1.0821562193840926
aggregate K 6.412728832934094
aggregate cons 1.5404171910709286
aggregate inc 2.0534354977056806
R 1.035354424234164
agg saving rate 24.9834147314562
